In [1]:
## SAVE DISTRIBUTED MODULE PROPERLY

In [1]:
import os
import torch
from resnet import ResNet

In [7]:
img_type = 'rgb'
time_stamp = '20200329_1239'
network_depth = 50
epoch = 154
learning_rate = 5e-05

root_work_dir = './Work_dirs/work_dirs_external/'
work_dir = root_work_dir+ img_type +'/' + time_stamp



model = ResNet(image_channels=3, num_classes=9, depth=network_depth)

model = torch.nn.DataParallel(model, device_ids = [0,1])
model.to(f'cuda:{model.device_ids[0]}')

pretrained_dict = torch.load(os.path.join(work_dir, 'model_best.pth.tar'))['state_dict'] #, map_location="cuda:0"
model.load_state_dict(pretrained_dict, strict=False)          
model = model.eval()

state_dict = model.module.state_dict()

optimizer = torch.optim.Adam(model.parameters(),learning_rate)

torch.save(
    {
                'epoch': epoch,
                'state_dict': state_dict,
                'optimizer_state_dict': optimizer.state_dict(),
            },
    os.path.join(work_dir, 'model_best2.pth.tar')
)


In [2]:
from utils import join_split_grids
from skimage import io
from sklearn.metrics import average_precision_score, precision_recall_curve,  PrecisionRecallDisplay
import torch
from torch import nn
import torchvision
import os
import matplotlib.pyplot as plt
import numpy as np
from resnet import ResNet, ResNetEnsembleInfraredRGB
from dataloaders import  load_sheep_grid_data, load_sheep_grid_multiband,load_sheep_grid_test_data, load_sheep_grid_multiband_test_data
from utils import to_cuda, get_grid, str2bool
from eval_utils import get_ensemble_model, do_predictions, write_results_to_file_and_calculate_AP
import time
import argparse
import json

img_type = 'ensemble'
time_stamp = '20200317_1321'


root_work_dir = './Work_dirs/work_dirs_external/'
work_dir = root_work_dir+ img_type +'/' + time_stamp
time_stamp_rgb = '20200225_1448'
time_stamp_infrared='20200312_2121'
network_depth=18
X=False
rgb_resize_shape=1024
infrared_resize_shape=64
fuse_depth=3


## Load Model ##
if img_type == 'ensemble':
    model = get_ensemble_model(root_work_dir,
                               time_stamp_rgb,
                               time_stamp_infrared,
                               network_depth,
                               X,
                               rgb_resize_shape,
                               infrared_resize_shape,
                               fuse_depth)
    
print(model)

ResNetEnsembleInfraredRGB(
  (rgb1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn